In [2]:
# 1. Initialization

from keras.preprocessing.image import ImageDataGenerator
from keras.applications.mobilenet_v3 import preprocess_input
from keras.applications.mobilenet_v3 import MobileNetV3Small
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras.optimizers import Adam
from keras.losses import CategoricalCrossentropy

import os


cwd = os.getcwd()

In [3]:
# 2. Creating data generators with data augmentation

path = os.path.join(cwd, "gallery")

datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    validation_split=0.4,
    width_shift_range=0.3,
    height_shift_range=0.3,
    horizontal_flip=True,
    vertical_flip=True
)
train_generator = datagen.flow_from_directory(path, subset='training', batch_size=16)
val_generator = datagen.flow_from_directory(path, subset='validation')

Found 40 images belonging to 16 classes.
Found 17 images belonging to 16 classes.


In [4]:
# 3. Loading model

base_model = MobileNetV3Small(weights="imagenet", include_top=False)
base_model.summary()

Model: "MobilenetV3small"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None, None,  0                                            
__________________________________________________________________________________________________
rescaling (Rescaling)           (None, None, None, 3 0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv (Conv2D)                   (None, None, None, 1 432         rescaling[0][0]                  
__________________________________________________________________________________________________
Conv/BatchNorm (BatchNormalizat (None, None, None, 1 64          Conv[0][0]                       
___________________________________________________________________________________

In [5]:
# 4. Expanding model

no_of_classes = train_generator.num_classes

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
out = Dense(no_of_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=out)
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None, None,  0                                            
__________________________________________________________________________________________________
rescaling (Rescaling)           (None, None, None, 3 0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv (Conv2D)                   (None, None, None, 1 432         rescaling[0][0]                  
__________________________________________________________________________________________________
Conv/BatchNorm (BatchNormalizat (None, None, None, 1 64          Conv[0][0]                       
______________________________________________________________________________________________

In [6]:
# 5. Configuring model for training

model.compile(optimizer=Adam(),
                  loss=CategoricalCrossentropy(),
                  metrics=["accuracy"])

In [8]:
# 6. Training model

# step_size_train = train_generator.n//train_generator.batch_size
model.fit(
    train_generator,
#     steps_per_epoch=step_size_train,
    epochs=5,
    validation_data=val_generator
)

Epoch 1/5
3/3 [==============================] - 3s 1s/step - loss: 0.0254 - accuracy: 1.0000 - val_loss: 0.2609 - val_accuracy: 0.9412
Epoch 2/5
3/3 [==============================] - 2s 748ms/step - loss: 0.0316 - accuracy: 0.9750 - val_loss: 0.1991 - val_accuracy: 0.9412
Epoch 3/5
3/3 [==============================] - 2s 911ms/step - loss: 0.0098 - accuracy: 1.0000 - val_loss: 0.3441 - val_accuracy: 0.9412
Epo

In [9]:
# 7. Saving model

model.save("model.h5")

/home/futuretribute/.local/lib/python3.8/site-packages/keras/utils/generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '
